## NeXtCortex project

The installation of the packages DrWatson, SpikingNeuralNetworks, UnPack, Logging, Plots, Statistics, Random, CSV, DataFrames, Unitful are required to run this file.

The file src/networks_utils.jl is also required.

In [44]:
using DrWatson
findproject(@__DIR__) |> quickactivate

include("src/network_utils.jl")
using .NetworkUtils

using SpikingNeuralNetworks
using UnPack
using Logging
using Plots
using Statistics
using Random
using CSV
using DataFrames
import SpikingNeuralNetworks: @update

global_logger(NullLogger())
SNN.@load_units

import SpikingNeuralNetworks: PoissonLayer, monitor!, sim!, SingleExpSynapse, IFParameter, PostSpike, STTC

img_path = "plots_and_images";

#### Network baseline configuration

In [ ]:
TC3inhib_network = (
    # Number of neurons in each population
    Npop = (TE=200, CE=4000, PV=400, SST=300, VIP=300),
    seed = 1234,

    # Parameters for cortical and thalamic excitatory neurons
    exc = IFParameter(
        τm=200pF / 10nS,  # Membrane time constant
        El=-70mV,         # Leak reversal potential
        Vt=-50.0mV,       # Spike threshold
        Vr=-70.0f0mV,     # Reset potential
        R=1 / 10nS,       # Membrane resistance
    ),

    # Parameters for the different populations of inhibitory neurons
    inh_PV  = IFParameter(τm=100pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),
    inh_SST = IFParameter(τm=200pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),
    inh_VIP = IFParameter(τm=150pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),

    # Spiking threshold properties: absolute refractory period
    spike     = PostSpike(τabs=5ms),
    spike_PV  = PostSpike(τabs=2ms),
    spike_SST = PostSpike(τabs=10ms),
    spike_VIP = PostSpike(τabs=5ms),

    # Synaptic properties
    synapse=SingleExpSynapse(
        τi=5ms,             # Inhibitory synaptic time constant
        τe=5ms,             # Excitatory synaptic time constant
        E_i=-80mV,          # Inhibitory reversal potential
        E_e=0mV             # Excitatory reversal potential
    ),
    synapse_PV  = SingleExpSynapse(τi=3ms, τe=5ms, E_i=-80mV, E_e=0mV),
    synapse_SST = SingleExpSynapse(τi=12ms, τe=5ms, E_i=-80mV, E_e=0mV),
    synapse_VIP = SingleExpSynapse(τi=7ms, τe=5ms, E_i=-80mV, E_e=0mV),

    # Connection probabilities and synaptic weights
    connections = (
        TE_to_CE = (p=0.05, μ=4nS, rule=:Fixed),
        TE_to_PV = (p=0.05, μ=4nS, rule=:Fixed),

        CE_to_CE = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_PV = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_TE = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_SST = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_VIP = (p=0.05, μ=2nS, rule=:Fixed),

        PV_to_CE  = (p=0.05, μ=10nS, rule=:Fixed),
        PV_to_PV  = (p=0.05, μ=10nS, rule=:Fixed),
        PV_to_SST = (p=0.05, μ=10nS, rule=:Fixed),

        SST_to_CE  = (p=0.025, μ=10nS, rule=:Fixed),
        SST_to_PV  = (p=0.025, μ=10nS, rule=:Fixed),
        SST_to_VIP = (p=0.025, μ=10nS, rule=:Fixed),

        VIP_to_SST = (p=0.3, μ=10nS, rule=:Fixed),
    ),

    # Parameters for external Poisson input
    afferents_to_TE=(
        layer=PoissonLayer(rate=1.5Hz, N=1000),           # Poisson input layer
        conn=(p=0.05, μ=4nS, rule=:Fixed),              # Connection probability and weight
    ),
    afferents_to_CE  = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.02, μ=4nS, rule=:Fixed)),
    afferents_to_PV  = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.02, μ=4nS, rule=:Fixed)),
    afferents_to_SST = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.15, μ=2nS, rule=:Fixed)),
    afferents_to_VIP = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.10, μ=2nS, rule=:Fixed)),
);

In [46]:
# Build and simulate
model = NetworkUtils.build_network(TC3inhib_network)
monitor!(model.pop, [:v], sr=1kHz)


Random.seed!(TC3inhib_network.seed)
sim!(model, 3s);

In [ ]:
rplt, zrplt, frplt, vplts = NetworkUtils.plot_analysis(model, img_path)

#Display of the baseline network activity
plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Baseline tight_layout.png")

#### Slower inhibition test (increasing PV membrane time constant)

In [ ]:
TC3inhib_network_modified = (; TC3inhib_network..., 
    synapse_PV  = SingleExpSynapse(τi=20ms, τe=5ms, E_i=-80mV, E_e=0mV),
)

model = NetworkUtils.build_network(TC3inhib_network_modified)
monitor!(model.pop, [:v], sr=1kHz)
Random.seed!(TC3inhib_network_modified.seed)
sim!(model, 3s)

rplt, zrplt, frplt, vplt = NetworkUtils.plot_analysis(model, img_path, name = "Slower inhibition")

plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Slower inhibition tight_layout.png")

#### Epileptic-like activity (increasing CE_to_CE)

In [ ]:
p_values = [0.15]
µ_values = [TC3inhib_network.connections.CE_to_CE.µ]
pops_to_modify = (:CE_to_CE,)
name = "Epileptic-like state"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

#### Thalamic increased connection (increasing TE_to_CE)

In [ ]:
p_values = collect(0.01:0.005:0.1)
pops_to_modify = (:TE_to_CE,)
name = "Thalamic increase"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
p_values = [0.08]
µ_values = [TC3inhib_network.connections.TE_to_CE.µ]
pops_to_modify = (:TE_to_CE,)
name = "Thalamic increase"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plots of the last p value

#### PV/SST loss (via genetic knockout) → less connection propability

PV loss

In [ ]:
include("src/network_utils.jl")
using .NetworkUtils

pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST)
p_values = [0.03, 0.02, 0.01, 0.0]
µ_values = [2, 5, 10]
name = "PV loss"

plt = NetworkUtils.heatmap_transition_time(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
p_values = collect(0.0:0.005:0.05)
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST)
name = "PV loss"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST)
p_values = [0.0]
µ_values = [10]
name = "PV loss"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

SST loss

In [ ]:
p_values = collect(0.0:0.005:0.5)
pops_to_modify = (:SST_to_CE, :SST_to_PV, :SST_to_VIP)
name = "SST loss"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
pops_to_modify = (:SST_to_CE, :SST_to_PV, :SST_to_VIP)
p_values = [0.0]
µ_values = [10]
name = "SST loss"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

PV and SST loss

In [ ]:
p_values = collect(0.0:0.005:0.5)
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST, :SST_to_CE, :SST_to_PV, :SST_to_VIP)
name = "PV_SST loss"

plt = NetworkUtils.plot_transition_time_vs_p(TC3inhib_network, p_values, pops_to_modify, name, img_path)
display(plt)

In [ ]:
pops_to_modify = (:PV_to_CE, :PV_to_PV, :PV_to_SST, :SST_to_CE, :SST_to_PV, :SST_to_VIP)
p_values = [0.03, 0.02, 0.01, 0.0]
µ_values = [2, 5, 10]
name = "PV_SST loss"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts) #display the plts of the last p and µ of p_values and µ_values

#### Optogenetic VIP inhibition → increases seizure threshold and shortens seizure duration.

VIP inhibition

In [ ]:
p_values = [0.0]
µ_values = [10]
pops_to_modify = (:VIP_to_SST,)
name = "VIP_inhibition"

plts = NetworkUtils.network_modifications(TC3inhib_network, p_values, µ_values, pops_to_modify, name, img_path)
display(plts)

#### Modulations Experiments

In [15]:
pops_to_modify = (:VIP_to_SST, :PV_to_CE, :SST_to_CE, :TE_to_CE)

μ_values = [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10]
p_values = [0.1, 0.2, 0.7, 1.0];

In [6]:
pops_to_modify = (:PV_to_CE, :TE_to_CE)

μ_values = [0.1, 1.0, 1.5]
p_values = [0.2, 0.7, 1.0];

In [ ]:
for pop in pops_to_modify

    csvfile = "$img_path/Modulation $pop sttc_results.csv"
    open(csvfile, "w") do io
        write(io, "mu,p,sttc\n")
    end

    for μ in μ_values, p in p_values

        modulation = (; TC3inhib_network.connections[pop]..., μ = μ, p = p)
        TC3inhib_network_modified = (; TC3inhib_network...,
            connections = (; TC3inhib_network.connections..., pop => modulation)
        )

        model = NetworkUtils.build_network(TC3inhib_network_modified)
        monitor!(model.pop, [:v], sr=1kHz)
        Random.seed!(TC3inhib_network_modified.seed)
        sim!(model, 3s)

        NetworkUtils.plot_analysis(model, img_path; name = "Modulation $pop", figs=false, csv = true, μ = μ, p = p)
    end

    df = CSV.read("$img_path/Modulation $pop sttc_results.csv", DataFrame)
    M = [ df[(df.mu .== μ) .& (df.p .== p), :sttc][1]
          for μ in μ_values, p in p_values ]

    sttc_heatmap = heatmap(
        p_values, μ_values, M,
        xlabel="p scale",
        ylabel="μ scale",
        title="Modulation $pop effect on synchrony",
        color = :viridis,
        clims = (0, 1) # to get the same color scale every time
    )

    savefig(sttc_heatmap, "$img_path/Modulation $pop sttc_heatmap.png")


    # Compute transition times with same threshold as sttc_heatmap
    T = [ NetworkUtils.average_transition_time(TC3inhib_network, pop, μ, p)
      for μ in μ_values, p in p_values ]

    transition_heatmap = heatmap(
        p_values, μ_values, T,
        xlabel = "p scale",
        ylabel = "μ scale",
        title  = "Transition time to synchrony for pop $pop",
        color  = :viridis,
        clims  = (0, 3),
        colorbar_title = "Time (s)"
    )

    savefig(transition_heatmap,
            "$img_path/Heatmap modulation $pop transition_time.png")

end